### Анализ экономических показателей регионов России

### На основании открытых данных, представленных ПАО Сбербанк, попробуем проанализировать представленные показатели о всех регионах России и как с годами меняются показатели на примере конкретного региона - Приморского края.

In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import base64
import io
import scipy.misc
import codecs
from IPython.display import HTML
from matplotlib_venn import venn2
from subprocess import check_output

from matplotlib.ticker import ScalarFormatter
import itertools
#import Translator
import time

In [27]:
data = pd.read_csv("DataSets/Economic parameters of Russian regions/df.csv", index_col="region", parse_dates=True)

In [28]:
data.head()

,name,date,value
region,,,
Russia,The number of applications for consumer loans,2013-01-15,571147
Russia,The number of applications for consumer loans,2013-02-15,504508
Russia,The number of applications for consumer loans,2013-03-15,628803
Russia,The number of applications for consumer loans,2013-04-15,836054
Russia,The number of applications for consumer loans,2013-05-15,867235


In [29]:
pd.unique(data['name'])

array(['The number of applications for consumer loans',
       'Average consumer loan application',
       'The number of applications for mortgages',
       'The average amount of a mortgage application',
       'Number of new deposits', 'Average amount of new deposit',
       'Average salary', 'Average pension',
       'On average rubles on current account per person',
       'On average, deposits in rubles per person',
       'Average spending on cards', 'Average Fast Food format Check',
       'Average check in Restaurant format',
       'Average spending in a fast food restaurant',
       'Average spending in a restaurant'], dtype=object)

In [30]:
data = data.pivot_table(index=["region", "date"], columns="name", values="value", aggfunc="mean", fill_value=0).reset_index()

### 1.2 Описание параметров

В получившейся таблице имеются следующие столбцы:

- Расходы
    - Average spending in a fast food restaurant', 'Average spending in a restaurant' - Средние расходы в фаст фуде/в ресторане
    - Average Fast Food format Check', 'Average check in Restaurant format' - Средний чек в фаст фуде/в ресторане
    - Average spending on cards' - Средние расходы по карте
- Доходы и сбережения
    - Average salary' - Средняя заработная плата
    - Average pension' - Средняя пенсия
    - On average rubles on current account per person' - В среднем рублей на расчетном счете на человека
    - On average, deposits in rubles per person' - В среднем размер вклада на человека
- Банковские операции
    - Потребительский кредит
        - The number of applications for consumer loans' - Количество заявок на потребительские кредиты
        - Average consumer loan application' - Средний размер потребительского кредита
    - Ипотека
        - The number of applications for mortgages' - Количество заявок на ипотеку
        - The average amount of a mortgage application' - Средний размер ипотеки
    - Депозиты
        - Number of new deposits' - Количество новых депозитов
        - Average amount of new deposit' - Средний размер депозита

In [44]:
data.head()

name,region,date,Average Fast Food format Check,Average amount of new deposit,Average check in Restaurant format,Average consumer loan application,Average pension,Average salary,Average spending in a fast food restaurant,Average spending in a restaurant,Average spending on cards,Number of new deposits,On average rubles on current account per person,"On average, deposits in rubles per person",The average amount of a mortgage application,The number of applications for consumer loans,The number of applications for mortgages,year
0,Altai region,2013-01-15,0,0,0,133509,0,0,0,0,0,0,0,0,968154,13423,508,2013
1,Altai region,2013-02-15,0,0,0,150030,0,0,0,0,0,0,0,0,957201,12440,641,2013
2,Altai region,2013-03-15,0,0,0,151795,0,0,0,0,0,0,0,0,974481,15906,853,2013
3,Altai region,2013-04-15,0,0,0,148752,0,0,0,0,0,0,0,0,985195,22172,1072,2013
4,Altai region,2013-05-15,0,0,0,150903,0,0,0,0,0,0,0,0,955750,22642,807,2013


In [40]:
data = data[data["region"] != "Russia"]

In [41]:
data['year'] = data["date"].apply(lambda t: time.strptime(t, '%Y-%m-%d').tm_year)

In [42]:
new_data = data[data["region"] == "Primorsky Krai"]
new_data = new_data.rename(columns={'Average check in Restaurant format' : 'Average check in Restaurant',
                                    'Average Fast Food format Check' : 'Average check in Fast Food'})

In [43]:
new_data.loc[new_data["year"]==2019, "year"] = 2018

### 1.3 Предобработка данных

### Удалим все строки с регионом "Russia", чтобы обобщённые по всей стране данные не создавали большие выбросы при сравнении с данными по отдельным регионам

In [58]:
data = data[data["region"] != "Russia"]

Добавим столбец год, чтобы было удобнее анализировать изменения признаков по годам.

In [59]:
data['year'] = data["date"].apply(lambda t: time.strptime(t, '%Y-%m-%d').tm_year)

Посмотрим, сколько записей представлено за каждый год. Из графика ниже видно, что вряд ли будет возможно делать выводы о показателях за 2019 год, так как за этот год слишком мало записей.

### 2. Анализ данных

### 2.1 Топ 20 регионов по различным показателям

Рассмотрим какие из регионов имеют самые высокие показатели по таким параметрам, как:

- Заработная плата
- Количество заявок на потребительские кредиты,
- Средний размер потребительских кредитов,
- Средние расходы по карте

и посмотрим есть ли между этими параметрами взаимосвязь.

### Средняя заработная плата

Самые высокие заработные платы в удалённых регионах Дальневосточного федерального округа таких, как Чукотский автономный округ, Магаданская область, Камчатский край и в Северных регионах таких, как Ямало-Ненецкий автономный округ, Ненецкий автономный округ, Ханты-Мансийский автономный округ, что имеет под собой такие основания, как удалённость регионов, тяжёлые климатические условия, дороговизна продуктов потребления. Помимо удалённых регионов вверху списка находятся Москва и Питер, что обусловлено статусами столиц у этих двух городов.

### Среднее количество заявок на потребительские кредиты¶

Явным лидером по количеству заявок на потребительские кредиты является Москва, после неё идут Краснодарский край и Московская область. Возможно, данный отрыв обусловлен более благоприятным уровнем жизни в данных регионах и поэтому население не боится брать кредиты.

### Размер потребительского кредита

Самые высокие размеры кредитов, как и заработные платы, в отдалённых Северных и Дальневосточных регионах, что обусловлено дороговизной жизни в этих регионах.

### Расходы по карте

В отдалённых регионах России уровень затрат у населения выше, чем в других регионах.

In [56]:
new_data = data[data["region"] == "Primorsky Krai"]
new_data = new_data.rename(columns={'Average check in Restaurant format' : 'Average check in Restaurant',
                                    'Average Fast Food format Check' : 'Average check in Fast Food'})

In [57]:
new_data.loc[new_data["year"]==2019, "year"] = 2018